In [ ]:
### Import Libraries.

import os
import scvelo as scv
import scanpy as sc
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


scv.settings.set_figure_params(dpi = 800, figsize = (6, 6))
scv.settings.verbosity = 3

In [ ]:
### Load Data.

os.chdir("/folder/")
adata = sc.read_h5ad("adata.h5ad")

In [ ]:
### RNA Velocity Estimation (dynamical model).

scv.tl.recover_dynamics(adata, n_jobs = 50)
scv.tl.velocity(adata, mode = 'dynamical')
scv.tl.velocity_graph(adata)

In [ ]:
### Velocity Plots.

scv.pl.velocity_embedding(
    adata,
    basis = 'umap',
    color = 'Cluster_Column',
    arrow_length = 2,
    arrow_size = 1.2,
    dpi = 800,
    legend_loc = 'right margin'
)

In [ ]:
### Velocity Plots.

scv.pl.velocity_embedding_stream(
    adata,
    basis = 'umap',
    color = 'Cluster_Column',
    density = 2,
    arrow_size = 1.2,
    dpi = 800,
    legend_loc = 'right margin'
)

In [ ]:
### Confidence Filtering.

scv.tl.velocity_confidence(adata)
scv.pl.scatter(adata, c = 'velocity_confidence', cmap = 'coolwarm', basis = 'umap')

adata_high_conf = adata[adata.obs['velocity_confidence'] > 0.6].copy()
print(f"{adata_high_conf.n_obs:,} cells retained after confidence filtering.")

In [ ]:
### Latent Time & Gene Dynamics.

scv.tl.latent_time(adata)
scv.pl.scatter(adata, color = 'latent_time', basis = 'umap', cmap = 'coolwarm')

top_genes = adata.var['fit_likelihood'].sort_values(ascending = False).index[:10]
scv.pl.heatmap(
    adata,
    var_names = top_genes,
    sortby = 'latent_time',
    col_color = 'Cluster_Column',
    n_convolve = 100,
    figsize = (6, 3)
)

In [ ]:
### Rank Dynamical Genes.

scv.tl.rank_dynamical_genes(adata, groupby = 'Cluster_Column')
ranked_genes = scv.get_df(adata, 'rank_dynamical_genes/names')
ranked_genes.to_excel("ranked_dynamical_genes.xlsx", index = False)

In [ ]:
### Feature-Level Velocity Visualization.

genes_of_interest = ["Feature_1", "Feature_2", "Feature_3", "Feature_4", "Feature_5"]

scv.pl.velocity(
    adata,
    var_names = genes_of_interest,
    basis = 'umap',
    color = 'Cluster_Column',
    ncols = 3,
    dpi = 200
)

In [ ]:
### Summary and Export

velocity_layer = adata.layers['velocity']
gene_means = np.array(velocity_layer.mean(axis = 0)).flatten()
velocity_summary = pd.DataFrame({
    'gene': adata.var_names,
    'mean_velocity': gene_means
})
velocity_summary.to_excel("velocity_mean_summary.xlsx", index = False)

In [ ]:
### Visualize Velocity Confidence Metrics.

scv.tl.velocity_confidence(adata)
scv.pl.scatter(
    adata,
    basis = "umap",
    color = ["velocity_length", "velocity_confidence"],
    cmap = "coolwarm",
    perc = [5, 95]
)

In [ ]:
### Summarize Velocity Metrics by Cluster_Column.

keys = ["velocity_length", "velocity_confidence"]
cluster_summary = adata.obs.groupby("Cluster_Column")[keys].mean().T
display(cluster_summary.style.background_gradient(cmap = "coolwarm", axis = 1))

In [ ]:
### BoxPlot.

category_order = [
    "Cluster_1",
    "Cluster_2",
    "Cluster_3",
    "Cluster_4"
]

custom_palette = {
    "Cluster_1": "#Color_1",
    "Cluster_2": "#Color_2",
    "Cluster_3": "#Color_3",
    "Cluster_4": "#Color_4"
}

df = (
    adata.obs[["velocity_length", "Cluster_Column", "Region"]]
    .dropna()
)
df = df[df["Cluster_Column"].isin(category_order)]
df["Cluster_Column"] = pd.Categorical(
    df["Cluster_Column"],
    categories = category_order,
    ordered = True
)

sns.set_style("whitegrid")
g = sns.catplot(
    data = df,
    x = "Cluster_Column",
    y = "velocity_length",
    kind = "box",
    height = 2.5,
    aspect = 2,
    hue = "Cluster_Column",
    col = "Region",
    palette = custom_palette,
    order = category_order,
    showfliers = False
)

g.set_axis_labels("Cluster_Column", "Velocity Magnitude")
g.set_titles(col_template = "Region: {col_name}")
plt.tight_layout()
plt.show()

In [ ]:
### Compute Pseudotime.

scv.tl.velocity_pseudotime(adata)

scv.pl.scatter(
    adata,
    color = "velocity_pseudotime",
    basis = "umap",
    cmap = "gnuplot",
    legend_loc = "right margin",
)

In [ ]:
### PAGA-based Velocity Connectivity.

adata.uns["neighbors"]["distances"] = adata.obsp["distances"]
adata.uns["neighbors"]["connectivities"] = adata.obsp["connectivities"]

scv.tl.paga(adata, groups = "Cluster_Column")

paga_df = scv.get_df(adata, "paga/transitions_confidence", precision = 2).T
paga_df.style.background_gradient(cmap = "Blues").format("{:.2g}")

In [ ]:
#### Kinetic Rate Parameters.

kinetic_genes = adata.var.query("fit_likelihood > 0.1 and velocity_genes == True")

kwargs = dict(xscale = "log", fontsize = 14)
with scv.GridSpec(ncols = 3) as pl:
    pl.hist(kinetic_genes["fit_alpha"], xlabel = "Transcription rate", **kwargs)
    pl.hist(kinetic_genes["fit_beta"] * kinetic_genes["fit_scaling"],
            xlabel = "Splicing rate", xticks = [.1, .4, 1], **kwargs)
    pl.hist(kinetic_genes["fit_gamma"], xlabel = "Degradation rate",
            xticks=[.1, .4, 1], **kwargs)

scv.get_df(adata, "fit*", dropna = True).head()

In [ ]:
### Differential Kinetic Testing.

genes_of_interest = ["Feature_1", "Feature_2", "Feature_3", "Feature_4", "Feature_5"]

scv.tl.differential_kinetic_test(
    adata,
    var_names = genes_of_interest,
    groupby = "Cluster_Column",
)

scv.get_df(
    adata[:, genes_of_interest],
    ["fit_diff_kinetics", "fit_pval_kinetics"],
    precision = 2,
)

scv.pl.scatter(
    adata,
    basis = genes_of_interest,
    linewidth = 2,
    add_linfit = True,
    frameon = False,
)

diff_clusters = list(adata[:, genes_of_interest].var["fit_diff_kinetics"])
scv.pl.scatter(
    adata,
    basis = "umap",
    add_outline = diff_clusters,
    outline_width = (0.8, 0.2),
)

In [ ]:
### Testing High-Likelihood Genes.

top_genes = adata.var["fit_likelihood"].sort_values(ascending = False).index[:100]
scv.tl.differential_kinetic_test(
    adata,
    var_names = top_genes,
    groupby = "Cluster_Column",
)
scv.pl.scatter(
    adata,
    basis = top_genes[:20],
    ncols = 5,
    add_outline = "fit_diff_kinetics",
    linewidth = 2,
    frameon = False,
)

In [ ]:
### Recompute Velocities (Differential Kinetics)

scv.tl.velocity(adata, diff_kinetics = True)
scv.tl.velocity_graph(adata)

scv.pl.velocity_embedding(
    adata,
    basis = "umap",
    color = "Cluster_Column",
    dpi = 800,
    arrow_size = 8,
    arrow_length = 20,
    legend_loc = "right margin",
)

In [ ]:
### Gene Velocity and Dynamics Profiles.

scv.tl.velocity_confidence(adata, vkey = "dynamical_velocity")

scv.pl.hist(
    [adata.obs["dynamical_velocity_confidence"].values],
    labels = ["dynamical model"],
    kde = True,
    colors = ["purple"],
    bins = 200,
    xlim = [0, 1],
)

scv.pl.velocity_embedding(
    adata,
    vkey = "dynamical_velocity",
    basis = "umap",
    color = "Cluster_Column",
    scale = 0.1,
    width = 0.003,
    frameon = False,
)

scv.pl.velocity_embedding_stream(
    adata,
    basis = "umap",
    color = "Cluster_Column",
    density = 2,
    arrow_size = 1.2,
    legend_loc = "right margin",
)

In [ ]:
### Visualize gene-specific Kinetic Profiles.

dm = scv.tl.recover_dynamics(adata, var_names = "Feature_1", max_iter = 1000)
dm.plot_profiles(fontsize = 14, dpi = 800)